In [4]:
import requests
import sys, os
from dotenv import load_dotenv
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from security_routes.utilities import hash_password
from pathlib import Path

## Base URL

You can run this notebook locally, on main runner, or development runner, please uncomment the one you are going to use, and comment the other base url's

In [5]:
# Main runner
# base_url = "https://tmv9bz5v4q.us-east-1.awsapprunner.com/"
# Development runner
# base_url = "https://cp3by92k8p.us-east-1.awsapprunner.com/"
# Local
base_url = "http://localhost:8000/"
prefix = "latest"

In [6]:
hash_password("password")

'$2b$12$xAY0w69jWoGB.SfdcoxOGucEi0mD1n3GXFr2dVTHFcyk016d3nkUq'

In [7]:
load_dotenv("../.env", override=True)

True

In [86]:
adminpassword = os.getenv("ADMIN_PASSWORD")
adminpassword is not None

True

In [98]:
response = requests.post(
    base_url+"latest/token", data={"username": "admin", "password": adminpassword}
)

admin_token = response.json()["access_token"]

In [99]:
len(admin_token)

145

Create user and group

In [100]:
new_user_data = {
    "username": "new_user",
    "email": "new_user@example.com",
    "password": "newpassword",
    "is_admin": False,
}

response = requests.post(
    base_url+f"{prefix}/users",
    params=new_user_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)

response

<Response [200]>

In [101]:
response.text

'{"id":4,"username":"new_user","email":"new_user@example.com","is_admin":false,"password":null}'

In [102]:
group_data = {
    "name": "new_group",
    "description": "A new group",
}
response = requests.post(
    base_url+f"{prefix}/groups",
    params=group_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [200]>

In [103]:
link_data = {
    "username": "new_user",
    "groupname": "new_group",
}
response = requests.post(
    base_url+f"{prefix}/link-user-group",
    params=link_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [201]>

Create a revision

In [104]:
response = requests.post(
    base_url+"latest/token", data={"username": "new_user", "password": "newpassword"}
)
token = response.json()["access_token"]

In [105]:
len(token)

151

In [106]:
new_version_data = {
    "name": "New Version",
    "iso_language": "eng",
    "iso_script": "Latn",
    "abbreviation": "NV",
    "rights": "Some Rights",
    "machineTranslation": False,
    "is_reference": True,
}

response = requests.post(
    base_url+f"{prefix}/version",
    json=new_version_data,
    headers={"Authorization": f"Bearer {token}"},
)
version_id =response.json()["id"]
response.json()

{'id': 2,
 'name': 'New Version',
 'iso_language': 'eng',
 'iso_script': 'Latn',
 'abbreviation': 'NV',
 'rights': 'Some Rights',
 'forward_translation_id': None,
 'back_translation_id': None,
 'machineTranslation': False,
 'owner_id': 4,
 'group_ids': [],
 'is_reference': True}

Update Name

In [107]:
# Modify name of the version
attr_update = {
    "id": version_id,
    "name": "New Version 2"
    }
response = requests.put(
    base_url+f"{prefix}/version",
    json=attr_update,
    headers={"Authorization": f"Bearer {token}"},
)
version_name = response.json()["name"]
version_name

'New Version 2'

In [108]:
print(new_version_data)

{'name': 'New Version', 'iso_language': 'eng', 'iso_script': 'Latn', 'abbreviation': 'NV', 'rights': 'Some Rights', 'machineTranslation': False, 'is_reference': True}


In [109]:
response.text

'{"name":"New Version 2","iso_language":"eng","abbreviation":"NV","forward_translation_id":null,"machine_translation":false,"deleted":false,"owner_id":4,"id":2,"iso_script":"Latn","rights":"Some Rights","back_translation_id":null,"is_reference":true,"deletedAt":null}'

In [110]:
headers = {"Authorization": f"Bearer {token}"}
test_revision = {
    "version_id": version_id,
    "name": "Test Revision",
}
test_upload_file = Path("../fixtures/eng-eng-kjv.txt")

with open(test_upload_file, "rb") as file:
    files = {"file": file}
    response = requests.post(
    base_url + f"{prefix}/revision", params=test_revision, files=files, headers=headers
    )
revision_id =response.json()["id"]
response.json()

{'id': 2,
 'bible_version_id': 2,
 'version_abbreviation': 'NV',
 'date': '2024-05-20',
 'name': 'Test Revision',
 'published': False,
 'back_translation_id': None,
 'machineTranslation': False,
 'iso_language': 'eng',
 'is_reference': False}

In [111]:
response.text

'{"id":2,"bible_version_id":2,"version_abbreviation":"NV","date":"2024-05-20","name":"Test Revision","published":false,"back_translation_id":null,"machineTranslation":false,"iso_language":"eng","is_reference":false}'

In [112]:
response = requests.get(
    base_url+f"{prefix}/revision",
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

[{'id': 1,
  'bible_version_id': 1,
  'version_abbreviation': 'BLTEST',
  'date': '2024-05-20',
  'name': None,
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language': 'eng',
  'is_reference': False},
 {'id': 2,
  'bible_version_id': 2,
  'version_abbreviation': 'NV',
  'date': '2024-05-20',
  'name': 'Test Revision',
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language': 'eng',
  'is_reference': False}]

In [113]:
response = requests.get(
    base_url+f"{prefix}/version",
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

[{'id': 2,
  'name': 'New Version 2',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  'abbreviation': 'NV',
  'rights': 'Some Rights',
  'forward_translation_id': None,
  'back_translation_id': None,
  'machineTranslation': False,
  'owner_id': 4,
  'group_ids': [3],
  'is_reference': True}]

Delete version and revision

In [114]:
response = requests.delete(
    base_url+f"{prefix}/revision",
    params={"id": revision_id},
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

{'detail': 'Revision 2 deleted successfully.'}

In [115]:
response = requests.delete(
    base_url+f"{prefix}/version",
    params={"id": version_id},
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

{'detail': 'Version New Version 2 successfully deleted.'}

Delete user and group

In [116]:
response = requests.delete(
    base_url+f"{prefix}/users",
    params={"username": "new_user"},
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [204]>

In [117]:
response = requests.delete(
    base_url+f"{prefix}/groups",
    params={"groupname": "new_group"},
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [204]>

In [118]:
headers = {"Authorization": f"Bearer {token}"}
response = requests.get(f"{base_url}/v3/language", headers=headers)
assert response.status_code == 200
assert response.json() == [{"iso639": "eng", "name": "english"}]

AssertionError: 